In [ ]:
# Project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from function import *

tranDtype = {'id': 'uint64',
 'chain': 'uint16',
 'dept': 'uint8',
 'category': 'uint16',
 'company': 'uint64',
 'brand': 'uint32',
#  'date' : 'object',
 'productsize': 'float32',
 'productmeasure': 'category',
 'purchasequantity': 'int64',
 'purchaseamount': 'float32'}

import pandas as pd

offers = pd.read_csv(dirThis + 'offers.csv')
transactions = pd.read_csv(dirThis + 'newdata.csv')
trainHistory = pd.read_csv(dirThis + 'trainHistory.csv')
testHistory = pd.read_csv(dirThis + 'testHistory.csv')

In [ ]:
from datetime import datetime
import time
import gc
file1 = dirThis + 'test/'

big_size = 20000000
k = 0
start = datetime.now()
tranCols = list(getTransactionsType().keys())
tt = getTransactionsType()

with pd.read_csv(dirThis + 'transactions.csv', chunksize = big_size, dtype = tt, 
                 header = None, skiprows = 1,
                #  parse_dates = ['date'], infer_datetime_format = True,
                 engine = 'c') as reader:
  for df in reader:
    if k > 0 :
      df = pd.concat([tmp, df], axis = 0, ignore_index = True)
    tmp = df.loc[df[0] == df[0].iloc[-1], :]
    df = df.loc[df[0] != df[0].iloc[-1], :]
    k += 1
    df.columns = tranCols
    use = generateFeature(offers, df, trainHistory, testHistory, k)
    path = file1 + f'testOut{str(k)}.csv'
    use.to_csv(path, index = False)

    del use
    gc.collect()
    print('Export : ', k, len(df), datetime.now() - start)
k += 1
tmp.columns = tranCols
use = generateFeature(offers, tmp, trainHistory, testHistory, k)
path = file1 + f'testOut{str(k)}.csv'
use.to_csv(path, index = False)
del use
gc.collect()
print('Export : (last one) ', k, len(df), datetime.now() - start)

In [ ]:
i = 1
total = k
split = pd.read_csv(file1 + f'testOut{str(1)}.csv', dtype = tt)
i += 1
for re in range(total-1) :
  split1 = pd.read_csv(file1 + f'testOut{str(i)}.csv', dtype = tt)
  i += 1
  split = pd.concat([split, split1], axis = 0, ignore_index = True)
  del split1
  gc.collect()

In [ ]:
split.shape

(311541, 185)

In [ ]:
split.to_csv(dirThis + 'featureNew/use.csv', index = False)